In [ ]:
from openai import OpenAI
import numpy as np
client = OpenAI(api_key="XXXXXXXXXXXXXXX")

response = client.embeddings.create(
    input="Pizza de massa fina assada em forno a lenha",
    model="text-embedding-3-small"
)

print("Embedding:", response.data[0].embedding)

Embedding: [-0.010012832470238209, -0.043606650084257126, -0.028226464986801147, -0.021233687177300453, 0.03663351386785507, -0.014113232493400574, -0.03392282873392105, 0.0034521929919719696, 0.024494364857673645, -0.04529591649770737, 0.018061403185129166, -0.005136549007147551, -0.041681669652462006, 0.01952477917075157, 0.010518630966544151, -0.013327527791261673, 0.01822836510837078, -0.022903311997652054, 0.007341434713453054, -0.06662781536579132, 0.011176658794283867, -0.03773350268602371, 0.002185243181884289, -0.01553732343018055, 0.04930301383137703, 0.019220318645238876, 0.0002673239796422422, -0.002749968785792589, 0.03150678426027298, 0.03415854275226593, -0.03569066524505615, -0.026655055582523346, 0.03268534317612648, -0.011137373745441437, 0.022726528346538544, -0.027578258886933327, -0.012973960489034653, -0.022353317588567734, 0.010047207586467266, 0.001937254797667265, -0.012168612331151962, 0.007071348372846842, 0.039442408829927444, 0.030210372060537338, 0.0827347

In [4]:
def get_embedding(text):
    response = client.embeddings.create(
        input=text,
        model="text-embedding-3-small"
    )
    return response.data[0].embedding

In [10]:
import pandas as pd
from tqdm import tqdm

data = pd.read_csv('cleaned_items_with_metadata.csv')
queries_df = pd.read_csv('./data/queries.csv')

item_texts = data["full_text"].tolist()
query_texts = queries_df["search_term_pt"].tolist()

In [11]:
text_embeddings = []
#with tqdm
for text in tqdm(item_texts, total=len(item_texts), desc="Processing items"):
    embedding = get_embedding(text)
    text_embeddings.append(embedding)

text_embeddings = np.array(text_embeddings)
np.save('text_embeddings_openai_small.npy', text_embeddings)


Processing items: 100%|██████████| 5000/5000 [30:28<00:00,  2.73it/s]   


In [12]:
batch_size = 32  # You can adjust based on API limits and performance
query_embeddings = []

for i in tqdm(range(0, len(query_texts), batch_size), desc="Processing query batches"):
    batch = query_texts[i:i+batch_size]
    response = client.embeddings.create(
        input=batch,
        model="text-embedding-3-small"
    )
    # Extract embeddings from each item in the response
    batch_embeddings = [item.embedding for item in response.data]
    query_embeddings.extend(batch_embeddings)

# Convert to numpy array and save
query_embeddings = np.array(query_embeddings)
np.save('query_embedding_openai_small.npy', query_embeddings)

Processing query batches: 100%|██████████| 4/4 [00:02<00:00,  1.57it/s]


In [14]:
quwery_embeddings = []
for query in tqdm(query_texts, total=len(query_texts), desc="Processing queries"):
    embedding = get_embedding(query)
    quwery_embeddings.append(embedding)

np.save('query_embedding_openai_small.npy', quwery_embeddings)

Processing queries: 100%|██████████| 100/100 [00:35<00:00,  2.82it/s]
